In [125]:
from web3 import Web3
import json 

w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/f00f7b3ba0e848ddbdc8941c527447fe'))
w3.isConnected()

True

In [126]:
wbtcoptions_address = "0x3961245DB602eD7c03eECcda33eA3846bD8723BD"
ercbtc_address  = "0x4CA6a755aFCFE55e8f2429D377D94cDAB8bCEf20"
btc_price_provider_address = "0x0275E872387E22Dbcb99Fc851b71eDe597214a21"

with open("/home/tom/Desktop/Medium/AutonomousHegician/fetch/HegicContractDeployer/contracts/btcoptions/build/contracts/HegicWBTCOptions.json", "r") as f:
    c = json.loads(f.read())

wbtc_option_contract = w3.eth.contract(address=wbtcoptions_address,
                                    abi=c["abi"])

with open("/home/tom/Desktop/Medium/AutonomousHegician/fetch/HegicContractDeployer/contracts/wbtc/build/contracts/FakeWBTC.json", "r") as f:
    c = json.loads(f.read())

wbtc_erc = w3.eth.contract(address=ercbtc_address,
                                    abi=c["abi"])

with open("/home/tom/Desktop/Medium/AutonomousHegician/fetch/HegicContractDeployer/contracts/btcpriceprovider/build/contracts/FakeBTCPriceProvider.json", "r") as f:
    c = json.loads(f.read())

price_provider = w3.eth.contract(address=btc_price_provider_address,
                                    abi=c["abi"])


with open("/home/tom/Desktop/Medium/AutonomousHegician/fetch/HegicContractDeployer/contracts/btcpool/build/contracts/HegicERCPool.json", "r") as f:
    c = json.loads(f.read())
    
pool_address = wbtc_option_contract.functions.pool().call()
pool = w3.eth.contract(address=pool_address,
                                    abi=c["abi"]
                      )



def toBTC(x):
    return int(w3.toWei(x, "ether") / 1e10)



In [59]:

period = 86400
amount =  toBTC(0.1)

strike_price = 200
# set the fees;
price_provider.functions.setPrice(strike_price).transact({'to': btc_price_provider_address,
                                                      'from': w3.eth.coinbase,
                                                     'value': w3.toWei(0.00, "ether")})


fees = wbtc_option_contract.functions.fees(period, amount, strike_price, 2).call()

print(fees)
#  we need to provide liqudity to the pool as so;
# we first mint our erc token




[261150, 0, 100000, 0, 161150]


In [60]:
pool_address

'0x3F074094d1fC1caBB6f415aCa9A00A30B3a2Df4D'

In [94]:
current_balance = wbtc_erc.functions.balanceOf("0xA16842b28FF96Ec695008996F0D85BE705A2c4Dd").call()
print("Current balance", current_balance)

wbtc_erc.functions.mint(amount).transact({'to': ercbtc_address,
                                           'from': "0xA16842b28FF96Ec695008996F0D85BE705A2c4Dd",
                                           'value':0})

current_balance = wbtc_erc.functions.balanceOf(w3.eth.coinbase).call()
print("Current balance", current_balance)

# now we have minted the amount, we need to go ahead and approve the transaction

# wbtc_erc.functions.approve(pool_address, amount).transact({
#                                            'from':  "0xA16842b28FF96Ec695008996F0D85BE705A2c4Dd",
#                                             "nonce": 241
#                                            })

Current balance 120000000
Current balance 30000000


In [91]:


pool.functions.provide(amount, 0).transact({'from': "0xA16842b28FF96Ec695008996F0D85BE705A2c4Dd",
                                           'value':0})


HexBytes('0x86f9c8176c5e31d9710c7d02308713edcbdb643ed45d61548c15357ddc1d595c')

In [68]:
wbtc_erc

'0x3F074094d1fC1caBB6f415aCa9A00A30B3a2Df4D'

In [112]:
wbtc_erc.functions.allowance( "0xA16842b28FF96Ec695008996F0D85BE705A2c4Dd", pool_address).call()

10000000

In [132]:
import pandas as pd






results = []


# we continue until we fail;
i = 0
try:
    while True:
        results.append(wbtc_option_contract.functions.options(i).call())
        i +=1
except Exception as e:
    print(f"Failed gathering option data. List exhausted? index = {i}")
    print(e)

Failed gathering option data. List exhausted? index = 115
Could not decode contract function call options return data b'' for output_types ['uint8', 'address', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint8']


In [133]:
pd.DataFrame(results, columns=["state", "holder", "strike", "amount", "lockedAmount", "premium", "expiration", "optionType"])


,state,holder,strike,amount,lockedAmount,premium,expiration,optionType
0,2,0xEb91FbD00444FcB9078030933A9beaB5a8C731FC,1135456371997,50000000,50000000,3498750,1604785049,2
1,1,0xC6f8Bcce8cc2f61A0323d211660120289283C9dE,20000000000000,180000000,180000000,715082,1604785228,2
2,3,0xC6f8Bcce8cc2f61A0323d211660120289283C9dE,20000000000000,200000000,200000000,149709,1602452623,2
3,1,0xC6f8Bcce8cc2f61A0323d211660120289283C9dE,20000000000000,690000000,690000000,2741147,1604785545,2
4,1,0x68924BC68Ff1357A0A08A20C64D538D9b9241574,1135500000000,100000000,100000000,6061074,1604181008,1
...,...,...,...,...,...,...,...,...
110,1,0x6481B4521Ad2Bb4b5ba4872e4a4D7829e5120eb0,1326602689430,20000000,20000000,234400,1604152876,2
111,1,0x82cCf5eD8F713281B1BE7d12F64A614e20c3BaD0,1344548000000,2000000000,2000000000,23440000,1604158620,2
112,1,0x091dDfcE101BeCd8556E863CA08C331Ba190C733,1353068429767,200000000,200000000,2344000,1604175530,2
113,1,0x091dDfcE101BeCd8556E863CA08C331Ba190C733,1353068429767,100000000,100000000,6220000,1606508744,2


In [ ]:
pd.to_csv("")